## Tune Vitrocal Parameters

In [19]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from vitrocal.datasets import catalog, ExcelDataset
from vitrocal.preprocessors import StandardPreprocessor
from vitrocal.detectors import DerivativeDetector, StandardExtractor
from vitrocal.analyzers import StandardAnalyzer

Pull out functions from `AnalyzeSingle.py` and work with them interactively

### Load example data file

In [20]:
def load_data(fpath: str | os.PathLike, load_args: dict={}) -> pd.DataFrame:
    """Load single neuron output file.

    Args:
        fpath (str | os.PathLike): Path to single Excel spreadsheet.
        load_args (dict, optional): Passed to `pd.read_excel()`. Defaults to None.

    Returns:
        pd.DataFrame: Dataframe
    """
    fname = os.path.basename(fpath)
    dataset = ExcelDataset.ExcelDataset(fpath, load_args)
    return dataset.load(), fname

In [ ]:
df, fname = load_data("./data/01_raw/E Green.xlsx")
# df, fname = load_data("../data/01_raw/E Green.xlsx")
df.head()

,1.000,22.640,14.464,81.846,22.303,5.206,13.008,20.742,14.349,17.126,...,4.038,19.026,6.572,17.669,7.024,2.347,23.887,42.383,4.802,7.620
0,2,21.593,13.584,81.234,19.634,4.756,10.206,17.576,14.729,16.779,...,3.390,19.026,9.156,16.792,7.568,2.142,25.340,41.388,4.140,7.929
1,3,20.348,13.595,81.041,22.261,5.335,9.868,13.821,14.625,16.988,...,4.179,18.215,7.307,17.280,7.383,2.056,27.561,41.679,3.816,6.994
2,4,22.938,13.941,81.420,21.579,5.410,9.857,13.642,14.422,16.672,...,3.982,19.623,7.675,16.628,7.272,2.002,23.300,41.452,3.906,7.850
3,5,20.889,12.441,81.222,20.250,5.081,11.525,12.387,13.078,16.708,...,4.201,19.561,5.383,20.522,7.516,1.527,19.236,46.553,4.277,7.748
4,6,21.691,14.711,81.017,21.015,5.553,11.386,17.477,13.776,16.634,...,3.926,19.548,5.114,18.126,7.079,1.802,23.300,45.417,4.044,7.735


Note: you want to be super careful about loading your files and looking at them first.
These files don't have a header, and sometimes they have numbered rows like this one.


In [ ]:
load_args = {'header': None, 'index_col': 0}
# df, fname = load_data("./data/01_raw/E Green.xlsx", load_args)
df, fname = load_data("./data/01_raw/E Green.xlsx", load_args)

`Vitrocal` is organized into three distinct modules for data analysis:
* `vitrocal.preprocessors`
* `vitrocal.detectors`
* `vitrocal.analyzers`

You'll want to explore the parameter space for each of these individually and assess
the impacts different parameter combinations have on your data.

See [the documentation](https://ajbarrow.w3.uvm.edu/assets/public_share/vitrocal_docs/api/) for details.

### Preprocess

In [23]:
# change these
fps = 1/2.5
bleach_period = 60
filter_frequency = None
baseline_threshold = 10 # percent
preprocess_window_size = 60 # seconds

# instantiate the StandardPreprocess object with these parameters
preprocessor = StandardPreprocessor(
        frames_per_second=1/2.5,
        bleach_period=bleach_period,
        filter_frequency=filter_frequency,
        baseline_threshold=baseline_threshold,
        window_size=preprocess_window_size
)

In [24]:
# call the object's `preprocess` method
# see the documentation for other methods available

processed = preprocessor.preprocess(df)

No filter applied.


## Detect and Extract

In [25]:
# change this
threshold = 20 # percent
detector = DerivativeDetector(threshold)

detected = detector.detect(processed)

## Extract 

Note: the `StandardExtractor` class has a `detect_and_extract` method

In [26]:
# change these
window = (3, 30) # seconds before and after
threshold = 20 # percent

extractor = StandardExtractor(
    window=window,
    frames_per_second=fps, # defined above
    threshold=threshold
)

events = extractor.extract(processed, detected)

With FPS = 0.4, a window of (3, 30) seconds captures 1 frame(s) before and 12 frame(s) after each event.


/Users/tony/Documents/phd/projects/other/vitrocal/vitrocal/detectors.py:137: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  identified[identified is False] = np.NaN # non-events


Note: this method returns events as a Python dictionary. You can examine individual events, if you like:

In [27]:
roi = 2
# events[roi]

## Analyze

In [28]:
# change these
upper_decay_bound = 0.8 # proportion
lower_decay_bound = 0.2 # proportion

analyzer = StandardAnalyzer(
    upper_decay_bound=upper_decay_bound,
    lower_decay_bound=lower_decay_bound
)

result, avg_result = analyzer.analyze(events)

In [29]:
result.head()

,roi,event,peak,upper,lower,decay
0,2,1.0,27.219529,21.297873269575085,2.423686623870037,18.874187
0,5,1.0,21.851080,6.71749433340203,-3.2533711260707254,9.970865
1,5,2.0,38.596155,15.092290988056458,1.5010871383174555,13.591204
2,5,3.0,37.804121,21.54067458187516,1.0369925070313724,20.503682
3,5,4.0,28.537583,17.316390264914933,NaN,NaN


In [30]:
avg_result.head()

,roi,total_events,average_peak,average_decay
0,2,1,27.219529,18.874187
1,5,8,35.123731,14.355113
2,6,20,52.596420,21.558414
3,7,29,67.141770,30.643565
4,13,6,29.688055,14.325707


In [ ]:
# save results
pd.to_pickle(events, "./data/02_output/demo_events.pkl") # python dictionary
pd.to_pickle(analyzer, "./data/02_output/demo_analyzer.pkl") # python object
result.to_csv("./data/02_output/demo_results.csv")
avg_result.to_csv("./data/02_output/demo_avg_results.csv")